In [2]:
# ignore for possible warnings
import warnings
warnings.filterwarnings('ignore')

# necessary packages
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random

# additional certain short functions
from torch import is_tensor
from matplotlib.pyplot import pause
from random import randint
from copy import deepcopy
from math import floor, ceil
from torch import stack
from torch import cat

# packages for data export and import
import pandas as pd
from PIL import Image
import os
import numpy as np

In [3]:
# get training dataset
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# get testing dataset
testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# class names
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# # print random item to visualize
# rand = random.randint(0, 59999)
# img = training_data.data[rand]
# plt.imshow(img, cmap='gray')
# print("ground truth = ", training_data.classes[training_data.targets[rand]])

# # print dataset size 
# size = len(training_data)
# print("Dataset size = ", size)

In [4]:
from data_generate import *

dataset = training_data
name = [
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
number = 1

test = pick(dataset, name, number)
test, number_list, stride_list = album(test, is_rand_stride = True, is_rand_pos = True)

print("length of number_lsit =", len(number_list))
print("sum of first 2 elements in number_list =", sum(number_list[:2]))
print("stride_list =", stride_list)

expansion shape = torch.Size([4, 84, 112])
temp shape = torch.Size([70, 84, 112])
numbers of each original album = [22, 13, 22, 13]
score shape = torch.Size([82, 84, 112])
numbers of each final album = [25, 15, 26, 16]
length of number_lsit = 4
sum of first 2 elements in number_list = 40
stride_list = [4, 7, 4, 7]


In [14]:
from data_io import *

data = dif_frame(test, number_list)
data_gt = vector(number_list, stride_list)

# print(data.shape)
# print(data_gt.shape)

# CSV_NAME = 'sample.csv'
# IMAGE_NAME = 'sample'

# gt_export(data_gt, CSV_NAME)
# gt = gt_import(CSV_NAME)
# image_export(data, IMAGE_NAME)


# testing path
PATH_DATA = os.path.join(os.getcwd(), "data\\test")

# export data testing
CSV_NAME = "samlpe.csv"
isExist = os.path.exists(PATH_DATA)
if not isExist:
    os.mkdir(PATH_DATA)
pd.DataFrame(data_gt).to_csv(os.path.join(PATH_DATA, CSV_NAME), index_label = "Ground Truth", header = ['x-vector'])

# import data testing
df = pd.read_csv(os.path.join(PATH_DATA, CSV_NAME))
gt = torch.Tensor(df.values[:, 1:]).reshape(len(df), 1)
print(gt.shape)

# export image testing
IMAGE_NAME = "samlpe.png"
array = data[2].numpy().astype(np.uint8)    # np.uint8 before Image.fromarray
im = Image.fromarray(array)
im.save(os.path.join(PATH_DATA, IMAGE_NAME), format="png")

# import image testing
img = Image.open(os.path.join(PATH_DATA, IMAGE_NAME))
print(np.array(list(img.getdata())).reshape(28*3, 28*4).shape)
# img.show()

torch.Size([78, 1])
(84, 112)


In [ ]:
# view image
for i in range(len(data)):
    plt.imshow(data[i], cmap='gray')
    pause(0.01)